In [1]:
import pandas as pd

In [2]:
#DB connection
import os
import json
wd = os.getcwd()
wd_list = wd.split("/")
unique_path = "/".join(wd_list[1:3])
#I have the passwords file saved as 'nothing_to_see_here.json'. Super secure, I know
path = "/"+unique_path+"/esh/nothing_to_see_here.json"
#Load json file in .esh and get into creds
#This accesses for the db_creds dictionary specifically. Other dicts are available by changing the ref below
creds = json.load(open(path))['db_creds']
from sqlalchemy import create_engine
###DONT SHARE WITH LOGIN DETAILS###
POSTGRES_ADDRESS = creds['esh_db_address']
POSTGRES_PORT =  creds['esh_db_port']
POSTGRES_USERNAME = creds['esh_db_username']
POSTGRES_PASSWORD = creds['esh_db_password']
POSTGRES_DBNAME = creds['esh_db_name']
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#This query gives the Total number of Households in the US and the number of Unconnected Households.
#also gives those totals in terms of populations

#using PUMS 2022 5Y estimates
q_dd='''
WITH unconnected AS
  (SELECT
    hh.state_id,
    hh.puma10_id,
    hh.puma20_id,
    hh.serialno,
    MAX(hh.wgtp) AS households,
    SUM(pop.pwgtp) AS population,

    CASE
      WHEN accessinet = '3'
        THEN TRUE
      ELSE FALSE END AS no_internet,
    CASE
      WHEN accessinet = '1' AND hispeed = '2' AND othsvcex = '2' AND dialup = '2' AND satellite = 
'2'
        THEN TRUE
      ELSE FALSE END AS cell_internet_only,
    CASE
      WHEN accessinet = '1' AND hispeed = '2' AND othsvcex = '2' AND dialup = '1' AND satellite = 
'2'
        THEN TRUE
      ELSE FALSE END AS dial_up_only
FROM
    dl.pums_households_2022 hh
    LEFT JOIN dl.pums_population_2022 pop
    ON hh.puma20_id = pop.puma20_id
    AND hh.puma10_id = pop.puma10_id
    AND hh.serialno = pop.serialno    
  WHERE hh.np>0 
  GROUP BY
    hh.state_id,
    hh.puma10_id,
    hh.puma20_id,
    hh.serialno,
    hh.accessinet,
    hh.hispeed,
    hh.othsvcex,
    hh.dialup,
    hh.satellite
  )
SELECT
  SUM(households) AS households,
  SUM(population) AS population,

  SUM(households) FILTER (WHERE no_internet = TRUE OR cell_internet_only = TRUE OR dial_up_only = 
TRUE) AS households_unconnected,
  SUM(population) FILTER (WHERE no_internet = TRUE OR cell_internet_only = TRUE OR dial_up_only = 
TRUE) AS population_unconnected

FROM
  unconnected

'''

dd=pd.read_sql(q_dd,cnx)
dd.head()

,households,population,households_unconnected,population_unconnected
0,126955900.0,334369976.0,26911718.0,57530183.0
